#### Creating tables and engine

In [343]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class School(Base):
    __tablename__ = "school"
    school_id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] 
    attendance_area_list: Mapped[List["Attendance_Area"]] = relationship(back_populates="school")
    def __repr__(self) -> str:
        return f"School(id={self.id!r}, name={self.name!r}, attendance area={self.attendance_area_list!r})"

class Attendance_Area(Base):
    __tablename__ = "attendance_area"
    id: Mapped[int] = mapped_column(primary_key=True)
    polygon_number: Mapped[int]
    lat_coordinate:Mapped[float]
    long_coordinate: Mapped[float]
    school_id = mapped_column(ForeignKey("school.school_id"))
    school: Mapped[School] = relationship(back_populates="attendance_area_list")
    def __repr__(self) -> str:
        return f"Attendance_Area(id={self.id!r}, polygon_number={self.polygon_number!r}, lat_coordinate={self.lat_coordinate!r}, long_coordinate={self.long_coordinate!r})"

In [346]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///test_orm.db", echo=True)

In [347]:
Base.metadata.create_all(engine)

2023-11-20 07:41:01,642 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-20 07:41:01,642 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("school")
2023-11-20 07:41:01,642 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-20 07:41:01,642 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("school")
2023-11-20 07:41:01,650 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-20 07:41:01,652 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("attendance_area")
2023-11-20 07:41:01,652 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-20 07:41:01,654 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("attendance_area")
2023-11-20 07:41:01,654 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-20 07:41:01,657 INFO sqlalchemy.engine.Engine 
CREATE TABLE school (
	school_id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (school_id)
)


2023-11-20 07:41:01,658 INFO sqlalchemy.engine.Engine [no key 0.00081s] ()
2023-11-20 07:41:01,668 INFO sqlalchemy.engine.Engine 
CREA

#### adding data individually

In [348]:
mkp = School(name='Ma Ko Pan Memorial College')
cuhk = School(name='CUHK')


In [349]:
a1 = Attendance_Area(polygon_number=1, lat_coordinate = 100, long_coordinate = -50, school = mkp)
a2 = Attendance_Area(polygon_number=1, lat_coordinate = 90, long_coordinate = -40, school = mkp)
a3 = Attendance_Area(polygon_number=2, lat_coordinate = 80, long_coordinate = -30, school = mkp)
a4 = Attendance_Area(polygon_number=2, lat_coordinate = 70, long_coordinate = -20, school = mkp)

b1 = Attendance_Area(polygon_number=1, lat_coordinate = 100, long_coordinate = -50, school = cuhk)
b2 = Attendance_Area(polygon_number=1, lat_coordinate = 90, long_coordinate = -40, school = cuhk)
b3 = Attendance_Area(polygon_number=2, lat_coordinate = 80, long_coordinate = -30, school = cuhk)
b4 = Attendance_Area(polygon_number=2, lat_coordinate = 70, long_coordinate = -20, school = cuhk)

In [350]:
from sqlalchemy.orm import Session
session = Session(engine)

In [351]:
session.add_all([mkp,a1,a2,a3,a4,cuhk,b1,b2,b3,b4])

In [352]:
session.flush()

2023-11-20 07:41:01,754 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-20 07:41:01,756 INFO sqlalchemy.engine.Engine INSERT INTO school (name) VALUES (?) RETURNING school_id
2023-11-20 07:41:01,758 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Ma Ko Pan Memorial College',)
2023-11-20 07:41:01,759 INFO sqlalchemy.engine.Engine INSERT INTO school (name) VALUES (?) RETURNING school_id
2023-11-20 07:41:01,760 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('CUHK',)
2023-11-20 07:41:01,763 INFO sqlalchemy.engine.Engine INSERT INTO attendance_area (polygon_number, lat_coordinate, long_coordinate, school_id) VALUES (?, ?, ?, ?) RETURNING id
2023-11-20 07:41:01,764 INFO sqlalchemy.engine.Engine [generated in 0.00014s (insertmanyvalues) 1/8 (ordered; batch not supported)] (1, 100.0, -50.0, 1)
2023-11-20 07:41:01,764 INFO sqlalchemy.engine.Engine INSERT INTO attendance_area (polygon_numbe

In [353]:
mkp.attendance_area_list

[Attendance_Area(id=1, polygon_number=1, lat_coordinate=100, long_coordinate=-50),
 Attendance_Area(id=2, polygon_number=1, lat_coordinate=90, long_coordinate=-40),
 Attendance_Area(id=3, polygon_number=2, lat_coordinate=80, long_coordinate=-30),
 Attendance_Area(id=4, polygon_number=2, lat_coordinate=70, long_coordinate=-20)]

In [354]:
b1.school_id

2

#### adding data by bulk

In [355]:
from sqlalchemy import insert

# Define your data:
schools_data = [
    {'name': 'Ma Ko Pan Memorial College'},
    {'name': 'CUHK'}
]




In [356]:
# Step 1: Insert school records and fetch their primary keys
stmt = insert(School).values(schools_data)
result = session.execute(stmt)
# Query for the inserted schools to get their primary keys
inserted_schools = session.query(School).filter(School.name.in_(['Ma Ko Pan Memorial College', 'CUHK'])).all()
school_ids = [school.school_id for school in inserted_schools]
print(school_ids)


2023-11-20 07:41:01,842 INFO sqlalchemy.engine.Engine INSERT INTO school (name) VALUES (?), (?)
2023-11-20 07:41:01,843 INFO sqlalchemy.engine.Engine [no key 0.00115s] ('Ma Ko Pan Memorial College', 'CUHK')
2023-11-20 07:41:01,846 INFO sqlalchemy.engine.Engine SELECT school.school_id AS school_school_id, school.name AS school_name 
FROM school 
WHERE school.name IN (?, ?)
2023-11-20 07:41:01,847 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ('Ma Ko Pan Memorial College', 'CUHK')
[1, 2, 3, 4]


In [357]:

# Step 2: Insert Attendance_Area records
attendance_areas_data = [
    {'polygon_number': 1, 'lat_coordinate': 100, 'long_coordinate': -50, 'school_id': school_ids[0]},
    {'polygon_number': 1, 'lat_coordinate': 90, 'long_coordinate': -40, 'school_id': school_ids[0]},
    {'polygon_number': 2, 'lat_coordinate': 80, 'long_coordinate': -30, 'school_id': school_ids[0]},
    {'polygon_number': 2, 'lat_coordinate': 70, 'long_coordinate': -20, 'school_id': school_ids[0]},
    {'polygon_number': 1, 'lat_coordinate': 100, 'long_coordinate': -50, 'school_id': school_ids[1]},
    {'polygon_number': 1, 'lat_coordinate': 90, 'long_coordinate': -40, 'school_id': school_ids[1]},
    {'polygon_number': 2, 'lat_coordinate': 80, 'long_coordinate': -30, 'school_id': school_ids[1]},
    {'polygon_number': 2, 'lat_coordinate': 70, 'long_coordinate': -20, 'school_id': school_ids[1]},
]
stmt = insert(Attendance_Area).values(attendance_areas_data)
result = session.execute(stmt)

2023-11-20 07:41:01,876 INFO sqlalchemy.engine.Engine INSERT INTO attendance_area (polygon_number, lat_coordinate, long_coordinate, school_id) VALUES (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?)
2023-11-20 07:41:01,878 INFO sqlalchemy.engine.Engine [no key 0.00200s] (1, 100.0, -50.0, 1, 1, 90.0, -40.0, 1, 2, 80.0, -30.0, 1, 2, 70.0, -20.0, 1, 1, 100.0, -50.0, 2, 1, 90.0, -40.0, 2, 2, 80.0, -30.0, 2, 2, 70.0, -20.0, 2)


In [358]:
from sqlalchemy import select
result = session.execute(select(School))

2023-11-20 07:41:01,907 INFO sqlalchemy.engine.Engine SELECT school.school_id, school.name 
FROM school
2023-11-20 07:41:01,908 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ()


In [359]:
'''for row in result:
    print(row)'''
session.rollback()

2023-11-20 07:41:01,932 INFO sqlalchemy.engine.Engine ROLLBACK


### Test data insert by loading data from csv

In [360]:
import pandas as pd
school_list = pd.read_csv('school_list.csv')

In [361]:
school_list.head(2)

,school_id,name,address,phone,fax,email,website,school_hour,grades,ward,...,grade_5_enrolment,grade_6_enrolment,grade_7_enrolment,grade_8_enrolment,grade_9_enrolment,grade_10_enrolment,grade_11_enrolment,grade_12_enrolment,attendance_area,walk_zone
0,142,A. E. Cross School,"3445 37 St SW Calgary, AB T3E 3C2",403-777-7410,587-933-9733,AECross@cbe.ab.ca,http://school.cbe.ab.ca/school/aecross,9:10am - 3:50pm,7-9,6,...,0,0,201,208,185,0,0,0,"['-114.1759081958474 51.071997543479668 0 0, -...","['-114.1179390176414 51.0173921233336 0 0, -11..."
1,108,Abbeydale School,"320 Abergale Dr NE Calgary, AB T2A 6W2",403-777-6970,587-933-9702,abbeydale@cbe.ab.ca,http://school.cbe.ab.ca/school/abbeydale,8:00 - 2:40,K-5,10,...,49,0,0,0,0,0,0,0,"['-113.9351192223485 51.066971217059027 0 0, -...","['-113.9351406030196 51.066972424404149 0, -11..."


In [362]:
import ast
school_list['attendance_area_conv'] = school_list['attendance_area'].apply(lambda x: ast.literal_eval(x))

In [363]:
sch_0 = school_list.iloc[0]

In [364]:
sch_0_data=[{
    'school_id': int(sch_0.school_id),
    'name':sch_0['name']
}]

In [365]:
sch_0_data

[{'school_id': 142, 'name': 'A. E. Cross School'}]

In [366]:
def parse_attendance_area(school):
    attendance_areas = []
    for i, polygon in enumerate(school.attendance_area_conv):
        for coordinate in polygon.split(','):
            lat, long = coordinate.split()[:2]
            attendance_areas.append({
                'school_id': school.school_id,
                'polygon_number': i,
                'lat_coordinate': float(lat),
                'long_coordinate': float(long)
            })
    return attendance_areas


In [367]:
sch_0_attendance_area = parse_attendance_area(sch_0)
print(len(sch_0_attendance_area))
sch_0_attendance_area[:5]

5618


[{'school_id': 142,
  'polygon_number': 0,
  'lat_coordinate': -114.1759081958474,
  'long_coordinate': 51.07199754347967},
 {'school_id': 142,
  'polygon_number': 0,
  'lat_coordinate': -114.1759052874672,
  'long_coordinate': 51.07119685412689},
 {'school_id': 142,
  'polygon_number': 0,
  'lat_coordinate': -114.1758802599086,
  'long_coordinate': 51.06834739140711},
 {'school_id': 142,
  'polygon_number': 0,
  'lat_coordinate': -114.175875022906,
  'long_coordinate': 51.06779152532258},
 {'school_id': 142,
  'polygon_number': 0,
  'lat_coordinate': -114.1762832040941,
  'long_coordinate': 51.06796095349047}]

Problem to be solved: the polygon number is lost in parsing, now there is only polygon_number 0 for all.

In [368]:
stmt1 = insert(School).values(sch_0_data)
stmt2=insert(Attendance_Area).values(sch_0_attendance_area)

In [369]:
session.execute(stmt1)

2023-11-20 07:41:02,901 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-20 07:41:02,902 INFO sqlalchemy.engine.Engine INSERT INTO school (school_id, name) VALUES (?, ?)
2023-11-20 07:41:02,903 INFO sqlalchemy.engine.Engine [no key 0.00078s] (142, 'A. E. Cross School')


In [370]:
session.execute(stmt2)

2023-11-20 07:41:03,383 INFO sqlalchemy.engine.Engine INSERT INTO attendance_area (polygon_number, lat_coordinate, long_coordinate, school_id) VALUES (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, 

In [371]:
result = session.execute(select(Attendance_Area))

2023-11-20 07:41:03,439 INFO sqlalchemy.engine.Engine SELECT attendance_area.id, attendance_area.polygon_number, attendance_area.lat_coordinate, attendance_area.long_coordinate, attendance_area.school_id 
FROM attendance_area
2023-11-20 07:41:03,439 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ()


In [372]:
for rows in result:
    print(rows)

(Attendance_Area(id=1, polygon_number=0, lat_coordinate=-114.1759081958474, long_coordinate=51.07199754347967),)
(Attendance_Area(id=2, polygon_number=0, lat_coordinate=-114.1759052874672, long_coordinate=51.07119685412689),)
(Attendance_Area(id=3, polygon_number=0, lat_coordinate=-114.1758802599086, long_coordinate=51.06834739140711),)
(Attendance_Area(id=4, polygon_number=0, lat_coordinate=-114.175875022906, long_coordinate=51.06779152532258),)
(Attendance_Area(id=5, polygon_number=0, lat_coordinate=-114.1762832040941, long_coordinate=51.06796095349047),)
(Attendance_Area(id=6, polygon_number=0, lat_coordinate=-114.1771896237207, long_coordinate=51.0683620621231),)
(Attendance_Area(id=7, polygon_number=0, lat_coordinate=-114.1785454427384, long_coordinate=51.06901705044878),)
(Attendance_Area(id=8, polygon_number=0, lat_coordinate=-114.1799049078428, long_coordinate=51.06974988619821),)
(Attendance_Area(id=9, polygon_number=0, lat_coordinate=-114.1810879743634, long_coordinate=51.070

In [373]:
#session.commit()

2023-11-20 07:41:03,834 INFO sqlalchemy.engine.Engine COMMIT


#### Rollback

In [374]:
#session.rollback()

#### Close session

In [375]:
session.close()